In [64]:
# Q1 42f05b9372a9a4a470db3b52817899b99a76ee73
# Q2 index 
# Q3 84.050095
# Q4 How do I copy files from a different folder into docker container’s working directory?
# Q5 1636 / 1462
# Q6 355 / 322
# Bonus 'To execute a command in a running Docker container, you first need to find the container ID using the command `docker ps`, and then you can execute a command in the specific container using the following syntax:\n\n```\ndocker exec -it <container-id> bash\n```\n\nThis will allow you to start an interactive bash session in the running container.'
# Bonus
# 150*1000 = 150000 inputs
# 250*1000 = 250000 outputs
# input cost = 150000 * 0.005 / 1000 = 0.75
# output cost = 250000 * 0.015 / 1000 = 3.75
# total cost  = 4.5

In [18]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [19]:
from openai import OpenAI
client = OpenAI()

In [68]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()

    return prompt

In [21]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": prompt}]
    )

    return response.choices[0].message.content

In [22]:
from tqdm.auto import tqdm
from elasticsearch import Elasticsearch

es_client = Elasticsearch("http://localhost:9200")

In [59]:
def elastic_search(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    results_docs = []
    
    for hit in response['hits']['hits']:
        results_docs.append(hit['_source'])     
    
    return results_docs

In [24]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "homework-1"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'homework-1'})

In [25]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:04<00:00, 230.88it/s]


In [93]:
query = "How do I execute a command in a running docker container?"

In [94]:
search_results = elastic_search(query)

In [95]:
prompt_reponse = build_prompt(query, search_results)

In [96]:
len(prompt_reponse)

1636

In [97]:
import tiktoken

In [98]:
encoding = tiktoken.encoding_for_model("gpt-4o")
encoding

<Encoding 'o200k_base'>

In [99]:
len(encoding.encode(prompt_reponse))

355

In [100]:
encoding.decode_single_token_bytes(63842)

b"You're"

In [101]:
llm(prompt_reponse)

'To execute a command in a running Docker container, you first need to find the container ID using the command `docker ps`, and then you can execute a command in the specific container using the following syntax:\n\n```\ndocker exec -it <container-id> bash\n```\n\nThis will allow you to start an interactive bash session in the running container.'